# My Titanic Project

## 0. Packages

In [2]:
# Handle table-like data and matrices calculation
import numpy as np
import scipy as sp
import pandas as pd


# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns


# Modelling Algorithms
# I will look at only four classification algorithms: LogisticRegression, GaussianNB, SVM, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier

# Modelling Helpers
from sklearn.preprocessing import Imputer , Normalizer , scale
# from sklearn.cross_validation import train_test_split , StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)

# 1. Get to know our dataset

In [3]:
train_df=pd.read_csv('train.csv',sep=',')
test_df=pd.read_csv('test.csv')

## Concatnate these two dataframe:
full_df=pd.concat([train_df,test_df])

In [4]:
full_df.head()

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,113803
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0.0,373450


In [7]:
print 'There are ',full_df.shape[1],' columns, and they are ',full_df.columns.values
print 'The total passengers are: ', full_df.shape[0]

There are  12  columns, and they are  ['Age' 'Cabin' 'Embarked' 'Fare' 'Name' 'Parch' 'PassengerId' 'Pclass'
 'Sex' 'SibSp' 'Survived' 'Ticket']
The total passengers are:  1309


Let's look at the missing values:

In [15]:
## This is a useful function:

def list_null(df):
    print df.shape[1],' columns:'
    for i in df.columns.values:
        print i,': nan',df[i].isnull().sum(),', ',df[i].dtypes
        
list_null(full_df)

12  columns:
Age : nan 263 ,  float64
Cabin : nan 1014 ,  object
Embarked : nan 2 ,  object
Fare : nan 1 ,  float64
Name : nan 0 ,  object
Parch : nan 0 ,  int64
PassengerId : nan 0 ,  int64
Pclass : nan 0 ,  int64
Sex : nan 0 ,  object
SibSp : nan 0 ,  int64
Survived : nan 418 ,  float64
Ticket : nan 0 ,  object
